In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import f_oneway

# Load datasets
summary_df = pd.read_csv("DataSets/DataSets/ev_city_station_summary.csv")
cities_df = pd.read_csv("DataSets/DataSets/canadacities.csv")
print(summary_df.head())

   Province        City  EV_Count  Charging_Stations
0   Ontario     toronto  250934.0                428
1   Ontario    hamilton   26036.0                 89
2    Quebec  sherbrooke   24964.0                100
3    Quebec    saguenay   16268.0                 16
4  Manitoba    winnipeg   12174.0                119


In [51]:
cities_df.columns


Index(['city', 'city_ascii', 'province_id', 'province_name', 'lat', 'lng',
       'population', 'density', 'timezone', 'ranking', 'postal', 'id'],
      dtype='object')

In [52]:
# Rename columns to standard names
cities_df = cities_df.rename(columns={
    "city": "City",
    "province_name": "Province",
    "population": "Population"
})

# Now merge with summary_df
merged = pd.merge(
    summary_df,
    cities_df[["City", "Province", "Population"]],
    on=["City", "Province"],
    how="left"
)
print(merged.head())

   Province        City  EV_Count  Charging_Stations  Population
0   Ontario     toronto  250934.0                428         NaN
1   Ontario    hamilton   26036.0                 89         NaN
2    Quebec  sherbrooke   24964.0                100         NaN
3    Quebec    saguenay   16268.0                 16         NaN
4  Manitoba    winnipeg   12174.0                119         NaN


In [53]:
# --- Clean after merge ---

# Replace missing Population with median
merged["Population"] = merged["Population"].fillna(merged["Population"].median())

# Replace missing EV_Count or Station counts with 0
merged["EV_Count"] = merged["EV_Count"].fillna(0)
merged["Charging_Stations"] = merged["Charging_Stations"].fillna(0)

# Avoid division by zero
merged["Charging_Stations"] = merged["Charging_Stations"].replace(0, 1)


c:\Users\ratho\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [54]:
# EV per capita
merged["EV_per_capita"] = merged["EV_Count"] / merged["Population"]

# Stations per capita
merged["Stations_per_capita"] = merged["Charging_Stations"] / merged["Population"]

# Distance proxy
merged["Distance_Score"] = 1 / (merged["Stations_per_capita"] + 1e-6)

# Accessibility: rural = 1, urban = 0
merged["Accessibility"] = np.where(merged["Population"] < 50000, 1, 0)
print(merged.head())

   Province        City  EV_Count  Charging_Stations  Population  \
0   Ontario     toronto  250934.0                428         NaN   
1   Ontario    hamilton   26036.0                 89         NaN   
2    Quebec  sherbrooke   24964.0                100         NaN   
3    Quebec    saguenay   16268.0                 16         NaN   
4  Manitoba    winnipeg   12174.0                119         NaN   

   EV_per_capita  Stations_per_capita  Distance_Score  Accessibility  
0            NaN                  NaN             NaN              0  
1            NaN                  NaN             NaN              0  
2            NaN                  NaN             NaN              0  
3            NaN                  NaN             NaN              0  
4            NaN                  NaN             NaN              0  


In [55]:
# Normalize features safely
for col in ["EV_per_capita", "Distance_Score", "Accessibility"]:
    merged[col + "_norm"] = (
        merged[col] - merged[col].min()
    ) / (merged[col].max() - merged[col].min() + 1e-9)


In [56]:
merged["RDI"] = (
    0.45 * merged["EV_per_capita_norm"] +
    0.35 * merged["Distance_Score_norm"] +
    0.20 * merged["Accessibility_norm"]
)


In [57]:
merged["RDI"] = merged["RDI"].replace([np.inf, -np.inf], np.nan)
merged["RDI"] = merged["RDI"].fillna(merged["RDI"].median())


c:\Users\ratho\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [58]:
features = ["EV_per_capita", "Distance_Score", "Accessibility"]
X = merged[features]
y = merged["RDI"]


In [59]:
"RDI" in merged.columns


True

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

lr = LinearRegression()
dt = DecisionTreeRegressor(max_depth=5)

lr.fit(X_train, y_train)
dt.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)
y_pred_dt = dt.predict(X_test)


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

results = pd.DataFrame({
    "Model": ["Linear Regression", "Decision Tree"],
    
    "MAE": [
        mean_absolute_error(y_test, y_pred_lr),
        mean_absolute_error(y_test, y_pred_dt),
    ],
    
    "RMSE": [
        np.sqrt(mean_squared_error(y_test, y_pred_lr)),   # FIXED
        np.sqrt(mean_squared_error(y_test, y_pred_dt)),   # FIXED
    ],
    
    "R2 Score": [
        r2_score(y_test, y_pred_lr),
        r2_score(y_test, y_pred_dt)
    ]
})

results


,Model,MAE,RMSE,R2 Score
0,Linear Regression,0.0,0.0,1.0
1,Decision Tree,0.0,0.0,1.0


In [61]:
import numpy as np

# -------------------------------------------------------
# 1. CREATE RAW FEATURES FOR RDI
# -------------------------------------------------------

# EV per capita
merged["EV_per_capita"] = merged["EV_Count"] / merged["Population"]

# Distance score (inverse availability of stations)
merged["Distance_Score"] = 1 / (merged["Charging_Stations"] + 1e-9)

# Accessibility score (Rural = 1, Urban = 0)
merged["Accessibility"] = np.where(merged["Population"] < 50000, 1, 0)


# -------------------------------------------------------
# 2. NORMALIZE ALL RDI FEATURES
# -------------------------------------------------------

def normalize(col):
    return (merged[col] - merged[col].min()) / (merged[col].max() - merged[col].min() + 1e-9)

merged["EV_per_capita_norm"] = normalize("EV_per_capita")
merged["Distance_Score_norm"] = normalize("Distance_Score")
merged["Accessibility_norm"] = normalize("Accessibility")


# -------------------------------------------------------
# 3. FINAL RDI FORMULA (Weights based on PDF)
# -------------------------------------------------------

merged["RDI"] = (
    0.45 * merged["EV_per_capita_norm"] +
    0.35 * merged["Distance_Score_norm"] +
    0.20 * merged["Accessibility_norm"]
)


# -------------------------------------------------------
# 4. CLEAN RDI VALUES
# -------------------------------------------------------

merged["RDI"] = merged["RDI"].replace([np.inf, -np.inf], np.nan)
merged["RDI"] = merged["RDI"].fillna(merged["RDI"].median())


# -------------------------------------------------------
# 5. SHOW TOP 10 UNDERSERVED AREAS (HIGH RDI)
# -------------------------------------------------------

top_underserved = merged.sort_values("RDI", ascending=False).head(10)[
    ["Province", "City", "EV_Count", "Charging_Stations", "Population", "RDI"]
]

top_underserved


c:\Users\ratho\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,Province,City,EV_Count,Charging_Stations,Population,RDI
0,Ontario,toronto,250934.0,428,NaN,NaN
1,Ontario,hamilton,26036.0,89,NaN,NaN
2,Quebec,sherbrooke,24964.0,100,NaN,NaN
3,Quebec,saguenay,16268.0,16,NaN,NaN
4,Manitoba,winnipeg,12174.0,119,NaN,NaN
5,Ontario,oshawa,12066.0,38,NaN,NaN
6,Ontario,london,12032.0,113,NaN,NaN
7,Quebec,drummondville,11234.0,45,NaN,NaN
8,Quebec,granby,10780.0,41,NaN,NaN
9,Quebec,saint-hyacinthe,8108.0,37,NaN,NaN
